In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 21.0 MB/s eta 0:00:00


In [3]:
import torch
import torch.nn.functional as F
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GCNConv, global_mean_pool

dataset = TUDataset(root='/tmp/PROTEINS', name='PROTEINS')

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 32)
        self.conv2 = GCNConv(32, 32)
        self.fc = torch.nn.Linear(32, dataset.num_classes)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN().to(device)
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

def train():
    model.train()
    total_loss = 0
    for data in data_loader:
        data = data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(data_loader)

def test():
    model.eval()
    correct = 0
    for data in data_loader:
        data = data.to(device)
        out = model(data)
        pred = out.argmax(dim=1)
        correct += pred.eq(data.y).sum().item()
    accuracy = correct / len(dataset)
    return accuracy

for epoch in range(200):
    loss = train()
    if epoch % 10 == 0:
        acc = test()
        print(f'Epoch: {epoch}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')

Processing...
Done!


Epoch: 0, Loss: 0.6765, Test Accuracy: 0.6181
Epoch: 10, Loss: 0.6426, Test Accuracy: 0.6927
Epoch: 20, Loss: 0.6341, Test Accuracy: 0.6676
Epoch: 30, Loss: 0.6321, Test Accuracy: 0.6667
Epoch: 40, Loss: 0.6186, Test Accuracy: 0.7080
Epoch: 50, Loss: 0.6345, Test Accuracy: 0.7044
Epoch: 60, Loss: 0.6286, Test Accuracy: 0.7134
Epoch: 70, Loss: 0.6245, Test Accuracy: 0.6604
Epoch: 80, Loss: 0.6274, Test Accuracy: 0.6855
Epoch: 90, Loss: 0.6319, Test Accuracy: 0.6676
Epoch: 100, Loss: 0.6255, Test Accuracy: 0.6739
Epoch: 110, Loss: 0.6342, Test Accuracy: 0.7098
Epoch: 120, Loss: 0.6290, Test Accuracy: 0.6891
Epoch: 130, Loss: 0.6276, Test Accuracy: 0.7053
Epoch: 140, Loss: 0.6294, Test Accuracy: 0.6990
Epoch: 150, Loss: 0.6252, Test Accuracy: 0.6846
Epoch: 160, Loss: 0.6287, Test Accuracy: 0.6891
Epoch: 170, Loss: 0.6311, Test Accuracy: 0.6972
Epoch: 180, Loss: 0.6285, Test Accuracy: 0.6990
Epoch: 190, Loss: 0.6304, Test Accuracy: 0.6505
